<h1 align="center">ELECTRE METHOD - PHASE I SHOW</h1>

<h3>This notebook illustrates how the <b>ELECTRE-II</b> method is used to rank the best tenders for one Procurement Procedure Lot.</h3>

The algorithm followed for this goes as follows:

1. Extract data from an Excel file containing the comparative values assigned by 3 evaluators to a set of Quality AwardCriteria
2. Transform the extracted data into a Graph compliant with the CAV-Anticorrupzione-Application Profile TBox 
3. Load the data in a Graph Store
4. Query the Graph Store to retrieve the data
5. Rank the tender-lots
6. Visualise the results

In [ ]:
import etl
import sparql as q
import store
import electre

<h2> Step #1: Extraction of the data from the excel file</h2>
<u><b>Note:</b></u>

If you want to change this file download it from <a href="./evaluation-tool.xlsm">here</a>, modify it and save it in the root directory of this notebook. 

In [ ]:
EXCEL_FILE = 'evaluation-tool.xlsm'
DATA_TAB = 'REPORTS PER EVALUATOR'
CRITERIA_TAB = 'MAIN REPORT'
g = etl.graph()  # Creates the graph of the ABox
g, criteria = etl.instantiate_criteria(g, EXCEL_FILE, CRITERIA_TAB)
e = etl.extract(EXCEL_FILE, DATA_TAB)           # Creates a pandas data frame from the Excel dataset

<h4>Here you have an overview of the data in a pandas' data frame:</h4>

In [ ]:
print(e)

<h2>Step #2: Creation of the CAV-AC-AP ABox</h2>

In [ ]:
g, r = etl.build_roles(g)                       # Instantiates the role 'evaluator'
g, tl = etl.lots_tendered(g)                    # Instantiates TenderLots 'A' to 'L'
g = etl.transform(e, g, tl, r, criteria)        # Populates the graph

<h4>The results have been extracted into a file named 'a-box.ttl'. Here you have the contents (it's a long process, since it generates many triples, so you will have to wait a short while):</h4>

In [ ]:
etl.show_save(g, True)

<h2>Step #3: Store the graph into the Graph Store</h2>

<h4>We first connect to the Graph Store.</h4>
<p><b><u>Note</u></b>: This show uses a local GraphDB store and a repository named 'cav-pilot'. In your case you will have to have a Graph Store available, either locally or remotely, and re-configure the code (see the function 'connect()' inside the 
module 'store.py'.</p>

In [ ]:
sparql = store.connect()

<h4>We then drop the current 'named graph', if any.</h4>

In [ ]:
store.drop(sparql)

<h4>Now we load the graph into the Graph Store (again, many triples are to be inserted, so wait a bit, please...)</h4>

In [ ]:
store.insert(g, sparql)

<h2>Step #4: Query the Graph Store and retrieve the data per criterion, evaluator, etc.</h4>

<h4>4.1. Let us retrieve now the data from the Graph Store.</h4> 

4.1.1. The <b><u>User Story</u></b> for this operation goes as this:

<p><div align="center"><i>I, as the algorithm calculating and ranking which is the winner of Lot1 of the procurement procedure X, want to get all input values introduced by all evaluators for this procedure's lot and concerning exclusively 'Qualitative Subjective Criteria', so I can assess the deviations between the evaluators' decisions.</i></div></p>

4.1.2. The <b><u>Competency Question</u></b> goes like this:
<p><div align="center"><i>Retrieve all the input values assigned by all evaluators to 'Qualitative Subjective Criteria', as well as the information related to the input value.</i></div></p>

4.1.3. And the corresponding <b><u>SPARQL Query</u></b> like this:

    PREFIX cap: <http://data.europa.eu/w21/2c930c7b-5e2f-4954-8522-bd3411339d6c/extension/cap#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX cccev: <https://data.europe.eu/semanticassets/ns/cv/cccev_v2.0.0#>
    PREFIX foaf: <http://xmlns.com/foaf/0.1/>
    PREFIX ct: <http://data.europa.eu/w21/2c930c7b-5e2f-4954-8522-bd3411339d6c/resource/criterion-type#>
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    SELECT 	?input (concat(?givenName, " ", ?familyName) as ?agent) ?source ?target ?winner ?value (str(?crit) as ?criterion) (str(?cObj) as ?cid) 
            ?maxscore ?note
    FROM cap:
    WHERE {
        ?input rdf:type cap:InputValue ;
           cap:comparesSourceThing ?source ;
           cap:comparesWithTargetThing ?target ;
           cap:hasBetterCandidate ?winner ;
           cap:hasNumericValue ?value ;
           cap:isProvidedBy ?agentObj;
           cap:refersToCriterion ?cObj .
        ?cObj cccev:hasName ?crit ; 
                   cap:hasMaximumScore ?maxscore ;
                   skos:scopeNote ?note .
        ?agentObj foaf:givenName ?givenName ;
                  foaf:familyName ?familyName .

        FILTER EXISTS {?cObj cccev:hasType ct:qs}
        FILTER (?source != ?target && ?value != 0)

    } ORDER BY ?agent

<h4>4.2. So, let us execute this SPARQL query...</h4>

In [ ]:
results = q._query(sparql)

4.2.1 The results are obtained as a ... <b>BIG</b> .. JSON object:

In [ ]:
print(results)

<h2>Step #5: Based on these results, we rank the tender-lots submitted by the tenderers</h2>

<h4>5.1. Let us get the J+ and P+, according to the ELECTRE-II, phase I algorithm:</h4>

In [ ]:
j = electre.get_j(results)
print(j)

This function returns a collection of criteria and per each criterion, it's <u>maximum score</u>, the <u>source tender lot</u> and the <u>target tender-lot</u> represented both with one letter (A-L, without J nor K), the <u>best tender-lot</u> resulting from comparing source and target, it's <u>sign</u> (+, = or -), the name of the <u>evaluator</u>, the <u>value</u> assigned by the evaluator as a result of the tender-pair-evaluation, and its weight (resulting from dividing its value by the maximum score).

Next we show you an example for one positive criterion:

In [ ]:
criterion = 'criterion-131d62d76f82880345bfb89aa8d8473b'
print('Criterion: {}'.format(criterion.split('-')[1]))
print('maximum score: {}'.format(j[criterion]['max_score']))
print('source tender-lot: {}'.format(j[criterion]['jw'][4][0]))
print('target tender-lot: {}'.format(j[criterion]['jw'][4][1]))
print('best tender-lot pair: {}'.format(j[criterion]['jw'][4][2]))
print('best tender-lot sign: {}'.format(j[criterion]['jw'][4][3]))
print('evaluator: {}'.format(j[criterion]['jw'][4][4]))
print('evaluator\'s assigned value : {}'.format(j[criterion]['jw'][4][5]))
print('relative weight (value / max_score) {}'.format(j[criterion]['jw'][4][6]))

<h4>5.2. With the individual J+ and P+ we can calculate the summation of each of the tender-lot weights per criteria.</h4>

<u><b>Note</b></u>: 
1. Remember that each tender-lot is represented by a letter from [A, L] without J nor K (do they not exist in Italian?).
2. The decimal number is the calculation of the P+ of all the criteria and all evaluators
3. Letters not printed out, except for J and K, are P= or P-, we are not interested in them, since P= is assigned a value '1', and P- a value of '0'.

In [ ]:
p = electre.get_p(j)
print(p)

<h4>5.3. Next we can check the <i>concordance</i> indices of each pair</h4>

<p>This calculation returns a JSON object with:</p>

1. the letter representing the best J+(a,a') tender-lot,
2. the decimal being the P+(a, a'), and 
3. the boolean telling us whether this tender-lot passes the threshold or not (True = pass, False= fails).


In [ ]:
# collect the concordance indices c(a, a') = P+(a, a') / P=(a, a') >= c, where c = 3/4 ('natural threshold')
# or c = 2/3 ('strong threshold')
# The decision of choosing 3/4 or 2/3 is set in the call for tenders.
c = 2/3
k = electre.get_c(p[0], c)
print(k)

<h4>5.4 Finally we rank the winners, sorted by score</h4>

In [ ]:
r = electre.rank(k)
for key in r:
    print('tender-lot: {0} -> score: {1} -> concordance threshold: {2}'.format(
        key[0], 
        round(key[1][0], 2), 
        'ABOVE (PASSES)' if key[1][1] else 'BELOW'))


<h2>6. Visualise the results</h2>

<h4>6.1. With these data one can start thinking about how to visualise the data...</h4>

e.g. a simple bar chart used to glimpse the differences between the tender-lots:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
x = []
y = []
threshold = 0.5
for key in r:
    x.append(key[0])
    y.append(round(key[1][0], 2))
    
plt.bar(x, y)
plt.xticks(x)
plt.plot([-1, len(x)], [0.5, .5], "k--")
plt.show()
             

<h4>6.2. Similar to the above, but as an area chart:</h4>

In [ ]:
plt.fill_between( x, y, color="skyblue", alpha=0.2)
p = plt.plot(x, y, color="Slateblue", alpha=0.6)

<h4>6.3. Other nice-to-have and TODO visualisations...for example:</h4>

* Spider chart per Quality Criterion and per tender-lot
* Standard deviation chart for all criteria, evaluators and tender-lots
* Other ...